In [1]:
import cv2
import numpy as np
import random

def copy_paste(src_img, bboxes, target_image, min_scale=0.8, max_scale=1.2):
    h, w = src_img.shape[:2]
    new_img = target_image.copy()
    # make the new_image has the same size as src_img
    new_img = cv2.resize(new_img, (w, h))
    new_boxes = []

    for (x1, y1, x2, y2) in bboxes:
        obj = src_img[y1:y2, x1:x2]

        # random scale
        scale = random.uniform(min_scale, max_scale)
        obj_resized = cv2.resize(obj, None, fx=scale, fy=scale)

        oh, ow = obj_resized.shape[:2]

        # random new position
        nx = random.randint(0, w - ow)
        ny = random.randint(0, h - oh)

        # Paste (soft blending)
        alpha = 0.7
        roi = new_img[ny:ny+oh, nx:nx+ow]
        new_img[ny:ny+oh, nx:nx+ow] = (roi * (1 - alpha) + obj_resized * alpha).astype(np.uint8)

        new_boxes.append([nx, ny, nx+ow, ny+oh])

    return new_img, new_boxes

In [ ]:
# create augmented dataset
import os

def augment_dataset(dataset_dir:str, augment_ratio:int = 1):
    images_dir = os.path.join(dataset_dir, 'images')
    labels_dir = os.path.join(dataset_dir, 'labels')

    image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
    label_files = [f.replace('.jpg', '.txt') for f in image_files]

    # get all idx for images have bounding boxes
    valid_indices = []
    for idx, label_file in enumerate(label_files):
        label_path = os.path.join(labels_dir, label_file)
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                if len(lines) > 0:
                    valid_indices.append(idx)
    
    # 
